In [1]:
## Pasos:

#1. RandomForest - MDI
#2. Seleccionar mejores caractéristicas.

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from sklearn.base import clone
plt.style.use('ggplot')
pd.set_option("display.max_columns", 50)

%matplotlib inline

In [13]:
#Lectura del dataset
file_name = "MI_database.csv"

df =  pd.read_csv(
    'data/02_data_intermedia/MI_preprocesado.csv', 
    header=0, 
    sep=","
)

In [14]:
df.head()

,SEX,AGE,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,...,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,TIME_B_S,NITR_S,NA_R_1_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,COMPLICATION_OUT
0,1,77.0,2.0,1.0,1.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15,0.15,8.000000,16.00000,4.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0
1,1,55.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.38,0.18,7.800000,3.00000,2.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0
2,1,52.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30,0.11,10.800000,13.44489,3.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
3,0,68.0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.37,8.782914,13.44489,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,4
4,1,60.0,0.0,0.0,0.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.45,0.22,8.300000,13.44489,9.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0


In [15]:
y = df['COMPLICATION_OUT']
X = df.copy()
del X['COMPLICATION_OUT']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
model = DecisionTreeClassifier(max_depth=2)



In [18]:
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [19]:
model.score(X_test, y_test)

0.4588235294117647

In [21]:
#rf = RandomForestRegressor(n_estimators=111)
rf = RandomForestRegressor(80, n_jobs=-1, oob_score=True)
#  40, n_jobs=-1, oob_score=True
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=80, n_jobs=-1, oob_score=True)

In [22]:
rf.feature_importances_

array([1.16712706e-02, 7.67050216e-02, 1.62057718e-02, 2.20788107e-02,
       9.86770161e-03, 2.04539694e-02, 1.09248616e-02, 2.54683277e-03,
       2.20847389e-02, 1.03039460e-02, 1.04642974e-02, 8.23901893e-03,
       3.01577046e-04, 2.69241580e-02, 1.36792378e-03, 6.62519095e-03,
       8.12433531e-08, 1.51232915e-07, 1.62727441e-06, 4.62750559e-04,
       5.13850543e-05, 3.25447295e-04, 2.47620637e-07, 1.74865997e-06,
       8.51509929e-03, 7.87481429e-03, 1.79221004e-03, 2.11241619e-03,
       1.38208375e-02, 4.18105453e-03, 2.15410076e-03, 3.99157853e-03,
       4.48143106e-02, 2.68903453e-02, 5.27098160e-03, 5.72392336e-04,
       7.75800465e-03, 1.11572979e-02, 6.63369199e-04, 9.06960304e-04,
       2.10608153e-02, 2.62561325e-02, 1.98044159e-02, 1.36478918e-02,
       8.98168410e-03, 9.23150489e-03, 3.38129408e-03, 2.09849711e-03,
       6.23259839e-04, 6.66075655e-03, 7.41567368e-03, 1.78241108e-02,
       2.61953401e-03, 7.40307436e-03, 6.63395018e-03, 4.01905646e-02,
      

In [23]:
importances = pd.DataFrame({"columns":X_train.columns, "importances":rf.feature_importances_}).sort_values("importances", ascending=False)

importances.style.bar()

,columns,importances
1,AGE,0.076705
80,L_BLOOD,0.065593
81,ROE,0.058232
78,ALT_BLOOD,0.052544
79,AST_BLOOD,0.050096
32,S_AD_ORIT,0.044814
55,n_r_ecg_p_05,0.040191
82,TIME_B_S,0.033184
84,NA_R_1_n,0.030553
13,nr_03,0.026924


In [27]:
# Se toman los valores mayores a 0.0009, que en total son 67 columnas 

In [25]:
X1 = X[['AGE','L_BLOOD','ROE','ALT_BLOOD','AST_BLOOD','S_AD_ORIT','n_r_ecg_p_05','TIME_B_S','NA_R_1_n','nr_03','D_AD_ORIT','lat_im','DLIT_AG','STENOK_AN','ant_im','IBS_POST','inf_im','n_r_ecg_p_01','INF_ANAM','ANT_CA_S_n','zab_leg_02','post_im','n_p_ecg_p_12','SEX','SVT_POST','GB','nr_11','ZSN_A','FK_STENOK','ritm_ecg_p_01','IM_PG_P','ASP_S_n','n_p_ecg_p_03','endocr_01','nr_01','NITR_S','TRENT_S_n','LID_S_n','endocr_02','MP_TP_POST','TIKL_S_n','ritm_ecg_p_08','n_r_ecg_p_03','ritm_ecg_p_07','n_r_ecg_p_04','nr_07','B_BLOK_S_n','O_L_POST','n_p_ecg_p_10','GEPAR_S_n','n_p_ecg_p_06','fibr_ter_03','zab_leg_03','n_p_ecg_p_07','zab_leg_06','ritm_ecg_p_02','n_r_ecg_p_02','SIM_GIPERT','zab_leg_04','zab_leg_01','ritm_ecg_p_04','fibr_ter_06','endocr_03','n_r_ecg_p_06','nr_04','n_p_ecg_p_09','FIB_G_POST']]

In [26]:
X1.shape

(1700, 67)

In [29]:
X1.to_csv('data/03_campos_priorizados/MI_priorizados.csv')